### This notebook combines, target for train dataset

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window as w
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

spark = (
    SparkSession.builder
    .config("spark.driver.memory","100g")
    .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.2.2")
    .config("fs.s3a.aws.credentials.provider","com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.driver.maxResultSize",0)
    # .config('spark.hadoop.io.compression.codecs', 'nl.basjes.hadoop.io.compress.SplittableGzipCodec')
    .getOrCreate()
)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.hadoop.io.compression.codecs", "org.apache.hadoop.io.compress.BZip2Codec")

s3_bucket_spark = "s3a://820323602090-team-dbad373c-7e36-407b-8690-05a44b804f43"
s3_bucket_pandas = "s3a://820323602090-team-dbad373c-7e36-407b-8690-05a44b804f43"

ModuleNotFoundError: No module named 'pyspark'

In [2]:
!aws s3 ls $team_bucket/data/hcp_universe/tremfya/

                           PRE negative/
                           PRE positive/


In [21]:
positive = spark.read.parquet(s3_bucket_spark+"/data/hcp_universe/tremfya/positive/").withColumn('target', f.lit(1))
negative = spark.read.parquet(s3_bucket_spark+"/data/hcp_universe/tremfya/negative/").withColumn('target', f.lit(0))

target = positive.union(negative).dropDuplicates()
target.show(5)

+-------+------------+-----------+----------------+------+
|   DRUG|PROVIDER_NPI| SPECIALTY1|      SPECIALTY2|target|
+-------+------------+-----------+----------------+------+
|Tremfya|  1447200845|Dermatology|            null|     1|
|Tremfya|  1629041728|Dermatology|            null|     1|
|Tremfya|  1306014550|Dermatology|            null|     1|
|Tremfya|  1578777017|Dermatology|            null|     1|
|Tremfya|  1124078902|Dermatology|Surgery, General|     1|
+-------+------------+-----------+----------------+------+
only showing top 5 rows



In [22]:
target.groupBy('target').count()

target,count
1,1853
0,10916


In [17]:
target.select('PROVIDER_NPI').distinct().count()

12769

In [24]:
target = target.select('PROVIDER_NPI', 'SPECIALTY2', 'target')

In [25]:
target.show(5)

+------------+----------------+------+
|PROVIDER_NPI|      SPECIALTY2|target|
+------------+----------------+------+
|  1447200845|            null|     1|
|  1629041728|            null|     1|
|  1306014550|            null|     1|
|  1578777017|            null|     1|
|  1124078902|Surgery, General|     1|
+------------+----------------+------+
only showing top 5 rows



In [27]:
target.write.parquet('../data/02_intermediate/train_target.parquet')
spark.stop()

In [32]:
import pandas as pd
tgt = pd.read_parquet('../data/02_intermediate/train_target.parquet/')
tgt

,PROVIDER_NPI,SPECIALTY2,target
0,1447200845,None,1
1,1629041728,None,1
2,1306014550,None,1
3,1578777017,None,1
4,1124078902,"Surgery, General",1
...,...,...,...
12764,1821059395,None,0
12765,1467566208,"Surgery, General",0
12766,1689833857,None,0
12767,1861727802,Family Practice,0


In [33]:
tgt.target.mean()

0.14511708042916438